# Tabular Data Analysis with `Pandas`

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

## Show first

### Dataset 1: Titanic

Let's start by loading the dataset as a Pandas **dataframe**:

In [ ]:
file_path = Path("./data/tabular/titanic.csv")
assert file_path.exists(), "The data file does not exist!"

In [ ]:
# Load the dataframe
df = pd.read_csv(file_path)

In [ ]:
# Take a look at the top 5 rows
df.head()

Let's check the age distribution of the passangers:

In [ ]:
_ = df["Age"].hist(bins=50)

Does the survival rate change between passenger sex?

In [ ]:
df.groupby("Sex")["Survived"].mean()

In [ ]:
# Or...
df.groupby("Sex")["Survived"].aggregate(lambda grp: grp.sum() / len(grp))

How does the survival rate differ between classes:

In [ ]:
_ = df.groupby("Pclass")["Survived"].mean().plot(kind="bar")

### Dataset 2: Air Quality Measurement Timeseries

AirBase: hourly measurements of all air quality monitoring stations from Europe.

In [ ]:
# Read the data
ts = pd.read_csv("./data/tabular/NO2_ts.csv", 
                 sep=";", 
                 skiprows=[1], 
                 na_values=["n/d"], 
                 index_col=0, 
                 parse_dates=True)

In [ ]:
# Check a few rows from the dataframe
ts.head()

Let's answer the following questions using Pandas:

**Does the air pollution shows a decreasing trend over the years?**

In [ ]:
# Plot monthly averages over the covered years
_ = ts["1999":].resample("M").mean().plot(ylim=[0, 120])

In [ ]:
# Same but on an annual basis
_ = ts["1999":].resample("A").mean().plot(ylim=[0, 100])

**What is the difference in diurnal profile between weekdays and weekend?**

In [ ]:
# Create the week day column
ts["weekday"] = ts.index.weekday

# Use the week day column to create the weekend binary column
ts["weekend"] = ts["weekday"].isin([5, 6])

# Group by the weekend(True/Falase)-hour and plot
ts_weekend = ts.groupby(["weekend", ts.index.hour])["BASCH"].mean().unstack(level=0)
_ = ts_weekend.plot()

## What is Pandas?

* **[Pandas](http://pandas.pydata.org/pandas-docs/stable/) can be thought of as NumPy arrays with labels for rows and columns**.
* Pandas provides a better support for heterogeneous data types, but it's also much, much more than that.
* Pandas can also be though of as `R`'s `data.frame` in Python.
* Pandas is powerful for working with:
    * Missing data.
    * Time-series data.
    * Reading and writing data.
    * Reshaping, grouping, and merging data.

So, when do we need Pandas?

In summary, we want to use Pandas when working with **tabular or structured data** (like SQL, excel, CSV, etc). We list the following sub-tasks:
- Importing data.
- Cleaning up messy data.
- Exploring data and gaining insights.
- Processing and preparing your data for machine learning.
- Data modeling together with scikit-learn, statsmodels, etc.

On the other hand, Pandas is not good with the following:

- Multi-dimensional array data: imagery, language, audio, etc.
- Labeled data (ex. climate data): have a look at [XArray](http://xarray.pydata.org/en/stable/).

## `DataFrame` & `Series`

A `Dataframe` is a tabular data structure comprised of rows and columns, akin to a spreadsheet, database tables, or R's data.frame objects. We can also think of it as multiple Series objects that share the same index.

In [ ]:
df

### Attributes of a `DataFrame`

A dataframe has besides an `index` attribute, the `columns` attribute:

In [ ]:
df.index

In [ ]:
df.columns

To check the data types of the different columns:

In [ ]:
df.dtypes

To get an overview of the dataframe we can use the `info()` method:

In [ ]:
df.info()

To get the dataframe's associated NumPy array, we can simply use `values`.

Attention: when you have heterogeneous data, all values will be upcasted:

In [ ]:
df.values

Apart from importing the data from an external source, one of the most common ways of creating a dataframe is from a dictionary with "column -> values" associations:

In [ ]:
# Define the dataset
data = {
    "country": ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
    "population": [11.3, 64.3, 81.3, 16.9, 64.9],
    "area": [30510, 671308, 357050, 41526, 244820],
    "capital": ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']
}

# Create a dataframe using the dataset
df_countries = pd.DataFrame(data)
df_countries

### `Series`: One-dimensional data

A series is a basic data holder for **one-dimensional labeled data**. 

In [ ]:
df["Age"]

In [ ]:
age = df["Age"]

### Attributes of a Series: `index` & `values`

The series has also an `index` and `values` attribute, but no `columns`:

In [ ]:
age.index

We can access the underlying NumPy array representation with the `.values` attribute:

In [ ]:
age.values[:10]

We can access the series values via the index, just like NumPy arrays:

In [ ]:
age[0]

Unlike NumPy arrays, a dataframe's index can be something other than integers:

In [ ]:
df = df.set_index("Name")
df

In [ ]:
# Get the ages indexed by name
age = df["Age"]

# Get the age of a specific person (indexing)
age["Dooley, Mr. Patrick"]

With the power of NumPy arrays, many things we can do with NumPy arrays can also be applied on DataFrames and Series:

In [ ]:
# Element-wise operations
age * 1000

In [ ]:
# ufuncs
age.mean()

In [ ]:
# Fancy indexing
age[age > 70]

.. but also a lot of Pandas sepcific methods, example:

In [ ]:
df["Embarked"].value_counts()

Let's answer a few questions:

What is the maximum fare that was paid? And the median?

In [ ]:
df["Fare"].max(), df["Fare"].median() 

Calculate the average survival ratio for all passengers:

In [ ]:
df["Survived"].mean()

## Data Import/Export

A wide range of input/output formats are natively supported by Pandas:

* CSV, text
* SQL database
* Excel
* HDF5
* json
* html
* pickle
* sas, stata
* (parquet)
* ...

In [ ]:
# Un-comment code -> go to end-of-line -> press tab 
#pd.read
#df.to

Pandas provides a very powerful CSV reader:

In [ ]:
pd.read_csv?

Luckily, if we have a well formatted CSV file, we won't need any of the "read_csv" arguments:

In [ ]:
df = pd.read_csv("./data/tabular/titanic.csv")
df.head()

## Exploration

Let's start by reading the pollution data:

In [ ]:
no2 = pd.read_csv("./data/tabular/NO2_ts.csv", 
                  sep=";", 
                  skiprows=[1], 
                  na_values=["n/d"], 
                  index_col=0, 
                  parse_dates=True)

We use `head` and `tail` to poke the dataframe:

In [ ]:
no2.head(3)

In [ ]:
no2.tail()

In [ ]:
# We use info() to get an overview
no2.info()

We can get some basic summary statistics about the data using `describe()`:

In [ ]:
no2.describe()

Pandas makes use of `matplotlib` internally which we can leverage to quickly visualize the data:

In [ ]:
no2.plot(kind="box", ylim=[0, 250])

Let's plot `BASCH`'s distribution:

In [ ]:
_ = no2["BASCH"].plot(kind="hist", bins=50)

The default plot is a line plot of all the columns:

In [ ]:
_ = no2.plot(figsize=(12, 6))

The dataframe is very dense, let's select a later slice of the dataframe: 

In [ ]:
_ = no2[-500:].plot(figsize=(12, 6))

## Selecting & Filtering Data

In [ ]:
df = pd.read_csv("./data/tabular/titanic.csv")

### `df[]` profiles convenient shortcuts

For a Dataframe, basic indexing selects the columns:

In [ ]:
# Select a single columns
df["Age"]

Or multiple columns:

In [ ]:
df[["Age", "Fare"]]

.. but slicing will access the rows instead:

In [ ]:
df[10:15]

### Systematic indexing with `loc` & `iloc`

When using `[]` like above, we can only select from axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:

- `loc`: selection by labels.
- `iloc`: selection by position.

In [ ]:
df = df.set_index("Name")

In [ ]:
# Select Miss. Elizabth Fare information
df.loc["Bonnell, Miss. Elizabeth", "Fare"]

In [ ]:
# Select all people between two names and get all columns
df.loc["Bonnell, Miss. Elizabeth":"Andersson, Mr. Anders Johan", :]

Selecting by position with `iloc` works similar to indexing NumPy arrays:

In [ ]:
# First two rows, second and third columns
df.iloc[:2, 1:3]

The different indexing methods can also be used to assign values:

In [ ]:
df.loc["Braund, Mr. Owen Harris", "Survived"] = 100
df.loc["Braund, Mr. Owen Harris"]

### Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with "boolean indexing" and comparable to NumPy. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed:

In [ ]:
df["Fare"] > 50

In [ ]:
# Filter
mask = df["Fare"] > 50
df[mask]

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Based on the titanic data set, select all rows for male passengers and calculate the mean age of those passengers. Do the same for the female passengers</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Based on the titanic data set, how many passengers older than 70 were on the Titanic?</li>
</ul>
</div>

## The "GroupBy" Operation

Let's start by creating some dummy data:

In [ ]:
df = pd.DataFrame({"key":["A", "B", "C", "A", "B", "C", "A", "B", "C"],
                   "data": [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Recap: Aggregating functions

When analyzing data, you often calculate summary statistics (aggregations like mean, max, min, etc). As we have seen before, we can easily calculate such statistics for a Series or column using one of the many available methods. For example:

In [ ]:
df["data"].sum()

However, in many cases, the data has certain groups in it, and in that case, you may want to calculate this statistic for each of the groups.

For example, in the above dataframe `df`, there is a column `key` which has three possible values: `A`, `B`, `C`. If we want to calcualte the sum for each of those groups, we could do the following:

In [ ]:
for k in list("ABC"):
    print(k, df[df["key"] == k]["data"].sum())

This becomes very verbose when having multiple groups. What we did above, apply a function on different groups, is a "groupby operation", and pandas provides some convenient functionalities to do this.

### GroupBy: applying functions per group

We want to **apply the same function on subsets of the dataframe, based on some key to split the dataframe in subsets**.

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:
1. **Splitting** the data into groups based on some criteria.
2. **Applying** a function to each group independently.
3. **Combining** the results into a data structure.

In [ ]:
# Group by unique values of "key" and sum over the rest of the columns
df.groupby("key").sum()

In [ ]:
# Same...
df.groupby("key").aggregate(np.sum)

### Application of the "groupby" concept on the titanic data 

We go back to the titanic passengers survival data:

In [ ]:
df = pd.read_csv("./data/tabular/titanic.csv")

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the average age for each sex again, but now using groupby.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the average survival ratio for all passengers.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate this survival ratio for all passengers younger that 25 (remember: filtering/boolean indexing).</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>What is the difference in the survival ratio between the sexes?</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Or how does it differ between the different classes? Make a bar plot visualizing the survival ratio for the 3 classes.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Make a bar plot to visualize the average Fare payed by people depending on their age. The age column is devided is separate classes using the `pd.cut` function as provided below.</li>
</ul>
</div>

## Working with Time-series data

In [ ]:
# Read the file into a dataframe
no2 = pd.read_csv("./data/tabular/NO2_ts.csv", 
                  sep=";", 
                  skiprows=[1], 
                  na_values=["n/d"], 
                  index_col=0, 
                  parse_dates=True)

Time-series related functionalities are enabled when the index is date-time. Let's make sure of that:

In [ ]:
no2.index

We can index the time-series using strings that represent dates:

In [ ]:
no2["2010-01-01 09:00": "2010-01-01 12:00"]

A nice feature is "partial date-string" indexing, e.g., we don't need to provide the full datetime string:

In [ ]:
# Get all data of January up to March 2012
no2["2012-01":"2012-03"]

Time and date components can be accessed from the index:

In [ ]:
no2.index.hour

In [ ]:
no2.index.year

### Converting time-series using `resample`

A very powerful Pandas method is `resample` that converts the frequency of the time-series (ex. hourly to daily).

In [ ]:
_ = no2.plot()

The time series has a frequency of 1 hour. Let's check it to daily:

In [ ]:
no2.resample("D").mean().head()

We can also get other statistics besides the mean:

In [ ]:
no2.resample("D").max().head()

The string to specify the new time frequency: http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases  
These strings can also be combined with numbers, eg `'10D'`.

<div class="alert alert-success">

<b>EXERCISE</b>: The evolution of the yearly averages with, and the overall mean of all stations

 <ul>
  <li>Use `resample` and `plot` to plot the yearly averages for the different stations.</li>
  <li>The overall mean of all stations can be calculated by taking the mean of the different columns (`.mean(axis=1)`).</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: how does the *typical monthly profile* look like for the different stations?

 <ul>
  <li>Add a 'month' column to the dataframe.</li>
  <li>Group by the month to obtain the typical monthly averages over the different years.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: The typical diurnal profile for the different stations

 <ul>
  <li>Similar as for the month, you can now group by the hour of the day.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: What is the difference in the typical diurnal profile between week and weekend days for the 'BASCH' station.

 <ul>
  <li>Add a column 'weekday' defining the different days in the week.</li>
  <li>Add a column 'weekend' defining if a days is in the weekend (i.e. days 5 and 6) or not (True/False).</li>
  <li>You can groupby on multiple items at the same time. In this case you would need to group by both weekend/weekday and hour of the day.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: What are the number of exceedances of hourly values above the European limit 200 µg/m3 ?

Count the number of exceedances of hourly values above the European limit 200 µg/m3 for each year and station after 2005. Make a barplot of the counts. Add an horizontal line indicating the maximum number of exceedances (which is 18) allowed per year?
<br><br>

Hints:

 <ul>
  <li>Create a new DataFrame, called `exceedances`, (with boolean values) indicating if the threshold is exceeded or not</li>
  <li>Remember that the sum of True values can be used to count elements. Do this using groupby for each year.</li>
  <li>Adding a horizontal line can be done with the matplotlib function `ax.axhline`.</li>
</ul>
</div>

## Things we have not covered

- Concatenating dataframes (`pd.concat`).
- Merging and joining data (`pd.merge`).
- Reshaping data (`pivot_table`, `melt`, `stack`, `unstack`).
- Working with missing data (`isnull`, `dropna`, `interpolate`).
- ...

# Further Reading

* Pandas documentation: http://pandas.pydata.org/pandas-docs/stable/
* Books
    * "Python for Data Analysis" by Wes McKinney
    * "Python Data Science Handbook" by Jake VanderPlas
* Tutorials (many good online tutorials!)
  * https://github.com/jorisvandenbossche/pandas-tutorial
  * https://github.com/brandon-rhodes/pycon-pandas-tutorial
* Tom Augspurger's blog
  * https://tomaugspurger.github.io/modern-1.html
  
# Credits

Joris Van den Bossche's [Pandas tutorial](https://github.com/jorisvandenbossche/pandas-tutorial).

---